In [5]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Install necessary packages
!pip install -q transformers
!pip install -q spacy
!pip install -q sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.5 MB/s eta 0:00:00


In [7]:
# Import required libraries
import os,shutil
from tqdm import tqdm
import pandas as pd
import numpy as np
import re
import random
import plotly.express as px
import plotly.graph_objs as go
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import torch

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(42)

Path of the folder containing transcript files

In [8]:
# Set the directory containing the transcription files
directory = "/content/drive/MyDrive/Transcription"

In [9]:
# Initialize a list to store file data
file_data = []

# Loop through each file in the directory for .text file
for file in os.listdir(directory):
    if file.endswith("txt"):
        file_path = os.path.join(directory, file)
        with open(file_path, "r") as f: 
            text_doc = f.read()
            docs = re.split(r'\n\n',text_doc) 
            team = file.split(".")[0]
            
            combined_docs = [] 
            current_doc = "" 
            for doc in docs: 
                if len(current_doc + doc) < 512:
                    current_doc += doc
                else:
                    combined_docs.append(current_doc) 
                    current_doc = doc
            combined_docs.append(current_doc)
            
            for i, doc in enumerate(combined_docs):  
                file_data.append({'Filenames': team, 'Texts': doc}) 

# Create a DataFrame from the file_data list
df = pd.DataFrame(file_data)
print(df.shape)
print(df.Filenames.unique())
df.head(10)

(353, 2)
['Audio A' 'Audio B' 'Audio C' 'Audio D' 'Audio E' 'Audio F' 'Audio G'
 'Audio J' 'Audio K' 'Audio L' 'Audio M' 'Audio H' 'Audio I']


,Filenames,Texts
0,Audio A,"Good morning, scholars. Good morning, everyone..."
1,Audio A,hero. This is a figure that I think is a lot m...
2,Audio A,come back from that. The only thing about his ...
3,Audio A,It's a little bit of a serious book. You got t...
4,Audio A,the Zook and the Dauphin. Probably the goofbal...
5,Audio A,is a very common character as well. What you l...
6,Audio A,works of literature then class. Drama or trage...
7,Audio A,little gems of wisdom. So the irony here is th...
8,Audio A,"higher ones, when they consider themselves bet..."
9,Audio A,of comments? How can he do it? How can he get ...


## T5

In [10]:
# Import necessary libraries
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion")
model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-base-finetuned-emotion")

In [11]:
# Create a deep copy (to create a different file) of the original DataFrame
df1 = df.copy(deep=True)
emotions = []
confidence = []

# Loop through each row in the DataFrame
for idx in tqdm(range(len(df1))): 
    speech = df1.iloc[idx,1] 
    try:
        input_ids = tokenizer.encode_plus(speech, max_length=512, truncation=True, padding='max_length', return_tensors='pt') 
        output_label = model.generate(input_ids=input_ids['input_ids'], attention_mask=input_ids['attention_mask'], max_length=2)
        dec = [tokenizer.decode(ids) for ids in output_label]
        label = re.sub('<pad>','',dec[0])
        emotions.append(label.strip())

        output = model(input_ids=input_ids['input_ids'], attention_mask=input_ids['attention_mask'], decoder_input_ids=torch.ones((1, 1)).long())
        logits = output.logits.squeeze(0)
        probs = torch.softmax(logits, dim=-1)
        max_value, max_index = torch.max(probs, dim=1)
        score = round(max_value.item(),6)
        confidence.append(score)

    except Exception as e:
        emotions.append("")
        confidence.append(0.0)

# Add the predicted emotions and confidence scores to the DataFrame
df1["Emotions_T5"] = emotions
df1['Emotions_T5'].replace({'joy': 'happiness', 'love': 'disgust'}, inplace=True)
df1["Confidence_T5"] = confidence
df['Emotions_T5'] = df1['Emotions_T5']
df["Confidence_T5"] = df1["Confidence_T5"]

df1 = df1.groupby(['Filenames', 'Emotions_T5']).size().reset_index(name='Counts')
print(df.isnull().sum(),df.shape,df1.isnull().sum(),df1.shape)


100%|██████████| 353/353 [28:08<00:00,  4.78s/it]

Filenames        0
Texts            0
Emotions_T5      0
Confidence_T5    0
dtype: int64 (353, 4) Filenames      0
Emotions_T5    0
Counts         0
dtype: int64 (43, 3)


In [12]:

# df['Emotions_T5'].replace({'joy': 'happiness', 'love': 'disgust'}, inplace=True)

In [13]:
df1.Emotions_T5.unique()

array(['anger', 'fear', 'happiness', 'sadness', 'surprise', 'disgust'],
      dtype=object)

In [14]:
# Define color scale for emotion categories
color_scale = {
    'disgust': 'whitesmoke',
    'happiness': 'springgreen',
    'surprise': 'torquise',
    'anger': 'crimson',
    'sadness': 'black',
    'fear': 'darkviolet'
}

fig = px.pie(df1, values='Counts', names='Emotions_T5', color='Emotions_T5', color_discrete_map=color_scale,
             labels={'Emotions_T5': 'Emotions_T5', 'Counts': 'Counts', 'Filenames': 'Filenames'},
             title='T5 Emotion Distribution by Filenames')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(showlegend=False)
fig.show()


In [15]:
# Create and display a 3D scatter plot of the T5 emotion distribution by filenames
fig = px.scatter_3d(df1, x='Filenames', y='Emotions_T5', z='Counts',size='Counts',
                    color='Emotions_T5',title='T5 Emotion Prediction by Filenames')
fig.show()

In [16]:
# Create and display a 3D scatter plot of the T5 emotion confidence distribution by filenames
fig = px.scatter_3d(df, x='Filenames', y='Emotions_T5', z='Confidence_T5',size='Confidence_T5',
                    color='Emotions_T5',title='T5 Emotion Confidence Score by Filenames')
fig.show()

In [17]:
# Create and display a histogram of the T5 emotion distribution by filenames
fig = px.histogram(df, x="Emotions_T5", color="Filenames", title="T5 Emotion Prediction by Filenames",
                   labels={"Emotions_T5": "Emotions", "Counts": "Counts"}, width=800, 
                   category_orders={"Emotions_T5": df['Emotions_T5'].value_counts().index})

fig.update_layout(barmode="group", bargap=0.1, bargroupgap=0, xaxis={'showgrid': False, 'zeroline': False})

fig.show()

## BERT

In [18]:
# Import necessary libraries and models
from transformers import RobertaTokenizerFast, TFRobertaForSequenceClassification, pipeline
from tqdm import tqdm
import tensorflow as tf

tokenizer = RobertaTokenizerFast.from_pretrained("arpanghoshal/EmoRoBERTa")
model = TFRobertaForSequenceClassification.from_pretrained("arpanghoshal/EmoRoBERTa")
emotion = pipeline('sentiment-analysis', model='arpanghoshal/EmoRoBERTa')


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [19]:
# Initialize lists to store emotions and confidence values
emotions = []
confidence = []

df1 = df.copy(deep=True)
for idx in tqdm(range(len(df1))):
    speech = df1.iloc[idx,1]
    input_ids = []
    attention_masks = []
    for i in range(0, len(speech), 512):
        chunk = speech[i:i+512]
        encoded = tokenizer.encode_plus(chunk, add_special_tokens=True, max_length=512, truncation=True, padding='max_length', return_attention_mask=True, return_tensors='tf')
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    
    if len(input_ids) > 1:
        input_ids = tf.concat(input_ids, axis=0)
        attention_masks = tf.concat(attention_masks, axis=0)
    elif len(input_ids) == 1:
        input_ids = input_ids[0]
        attention_masks = attention_masks[0]
    else:
        emotions.append("")
        confidence.append(0.0)
        continue

    try:
        outputs = model({'input_ids': input_ids, 'attention_mask': attention_masks})[0]
        predicted_class = tf.argmax(outputs, axis=1).numpy()[0]
        predicted_scores = tf.nn.softmax(outputs, axis=1).numpy()[0]
        emotions.append(model.config.id2label[predicted_class])
        confidence.append(predicted_scores[predicted_class])
    except:
        emotions.append("")
        confidence.append(0.0)
    
mappings = {
    'neutral' : 'fear',
    'admiration': 'happiness',
    'amusement': 'happiness',
    'anger': 'anger',
    'annoyance': 'anger',
    'approval': 'happiness',
    'caring': 'sadness',
    'confusion': 'surprise',
    'curiosity': 'surprise',
    'desire': 'happiness',
    'disappointment': 'sadness',
    'disapproval': 'anger',
    'disgust': 'disgust',
    'embarrassment': 'disgust',
    'excitement': 'happiness',
    'fear': 'fear',
    'gratitude': 'happiness',
    'grief': 'sadness',
    'joy': 'happiness',
    'love': 'happiness',
    'nervousness': 'fear',
    'optimism': 'happiness',
    'pride': 'happiness',
    'realization': 'surprise',
    'relief': 'happiness',
    'remorse': 'sadness',
    'sadness': 'sadness',
    'surprise': 'surprise'
}

emotions = [*map(mappings.get,emotions)]
df1["Emotions_BERT"] = np.array(emotions)
df1["Confidence_BERT"] = np.array(confidence)
df['Emotions_BERT'] = df1['Emotions_BERT']
df['Confidence_BERT'] = df1['Confidence_BERT']
df1 = df1.groupby(['Filenames','Emotions_BERT','Confidence_BERT']).size().reset_index(name='Counts')
print(df.isnull().sum(),df.shape,df1.isnull().sum(),df1.shape)

100%|██████████| 353/353 [18:26<00:00,  3.13s/it]

Filenames          0
Texts              0
Emotions_T5        0
Confidence_T5      0
Emotions_BERT      0
Confidence_BERT    0
dtype: int64 (353, 6) Filenames          0
Emotions_BERT      0
Confidence_BERT    0
Counts             0
dtype: int64 (353, 4)


In [20]:
# Define a color scale for visualizations
color_scale = {
    'fear': 'whitesmoke',
    'happiness': 'springgreen',
    'surprise': 'torquise',
    'anger': 'crimson',
    'sadness': 'black',
    'disgust': 'darkviolet'
}

# Create and display a pie chart of the BERT emotion distribution by filenames
fig = px.pie(df1, values='Counts', names='Emotions_BERT', color='Emotions_BERT', 
             color_discrete_map=color_scale,color_discrete_sequence=px.colors.qualitative.Pastel,
             labels={'Emotions_BERT': 'Emotions_BERT', 'Counts': 'Counts', 'Filenames': 'Filenames'},
             title='BERT Emotion Distribution by Filenames')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(showlegend=False)
fig.show()

In [21]:
# Create and display a 3D scatter plot of the BERT emotion distribution by filenames
fig = px.scatter_3d(df1, x='Filenames', y='Emotions_BERT', z='Counts',size='Counts',
                    color='Emotions_BERT',title='BERT Emotion Prediction by Filenames')
fig.show()

In [22]:
# Create and display a 3D scatter plot of the BERT emotion confidence distribution by filenames
fig = px.scatter_3d(df, x='Filenames', y='Emotions_BERT', z='Confidence_BERT',size='Confidence_BERT',
                    color='Emotions_BERT',title='BERT Emotion Confidence Score by Filenames')
fig.show()

In [23]:
# Create and display a histogram of the BERT emotion distribution by filenames
fig = px.histogram(df, x="Emotions_BERT", color="Filenames", title="BERT Emotion Distribution by Filenames",
                   labels={"Emotions_BERT": "Emotions", "Counts": "Counts"}, width=800, 
                   category_orders={"Emotions_BERT": df['Emotions_BERT'].value_counts().index})

fig.update_layout(barmode="group", bargap=0.1, bargroupgap=0, xaxis={'showgrid': False, 'zeroline': False})

fig.show()

## XLNet

In [24]:
# Import necessary libraries and models
from transformers import XLNetTokenizer, TFXLNetForSequenceClassification, pipeline

tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
model = TFXLNetForSequenceClassification.from_pretrained('xlnet-base-cased')

/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning:

The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.

Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetForSequenceClassification: ['lm_loss']
- This IS expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassifica

In [25]:
# Create a copy of the original dataframe
df1 = df.copy(deep=True)
emotions_probs = []
emotion_labels = []

for idx in tqdm(range(len(df1))):
    speech = df1.iloc[idx,1]
    try:
        input_ids = tokenizer.encode(speech,add_special_tokens=True,max_length=512,truncation=True,return_tensors='tf')
        output = model(input_ids=input_ids, training=False)
        dec = output.logits
        label = tf.argmax(dec, axis=1).numpy()[0]
        if label==0:
            emotions_probs.append(dec[0][0].numpy())
        else:
            emotions_probs.append(dec[0][1].numpy())
        label_pos_neg = {0:"negetive",1:"positive"}
        emotion_labels.append(label_pos_neg[label])
    except:
        emotion_labels.append("")
        emotions_probs.append(0.0)

# Add the predicted emotions and confidence values to the dataframe
df1["Emotions_XLNet"] = np.array(emotion_labels)
df1["Confidence_XLNet"] = np.array(emotions_probs)
df['Emotions_XLNet'] = df1['Emotions_XLNet']
df['Confidence_XLNet'] = df1['Confidence_XLNet']

df1 = df1.groupby(['Filenames','Emotions_XLNet','Confidence_XLNet']).size().reset_index(name='Counts')

print(df.isnull().sum(),df.shape,df1.isnull().sum(),df1.shape)

100%|██████████| 353/353 [06:12<00:00,  1.05s/it]

Filenames           0
Texts               0
Emotions_T5         0
Confidence_T5       0
Emotions_BERT       0
Confidence_BERT     0
Emotions_XLNet      0
Confidence_XLNet    0
dtype: int64 (353, 8) Filenames           0
Emotions_XLNet      0
Confidence_XLNet    0
Counts              0
dtype: int64 (353, 4)


In [26]:
# Define a color scale for visualizations
color_scale = {
    'positive': 'whitesmoke',
    'negative': 'crimson'
}

# Create and display a pie chart of the XLNet emotion detection confidence based on each emotion category
fig = px.pie(df1, values='Counts', names='Emotions_XLNet', color='Emotions_XLNet', color_discrete_map=color_scale,
             labels={'Emotions_XLNet': 'Emotions_XLNet', 'Counts': 'Counts', 'Confidence_XLNet': 'Confindence_XLNet'},
             title='XLNet Emotion Detection Confidance based on Each Emotion Category')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(showlegend=False)
fig.show()

In [27]:
# Create and display a 3D scatter plot of the XLNet emotion distribution by filenames
fig = px.scatter_3d(df1, x='Filenames', y='Emotions_XLNet', z='Counts',size='Counts',
                    color='Emotions_XLNet',title='XLNet Emotion Prediction by Filenames')
fig.show()

In [28]:
# Create and display a 3D scatter plot of the XLNet emotion confidence distribution by filenames
fig = px.scatter_3d(df, x='Filenames', y='Emotions_XLNet', z='Confidence_XLNet',
                    color='Emotions_XLNet',title='XLNet Emotion Confidence Score by Filenames')
fig.show()

In [29]:
# Create and display a histogram of the XLNet emotion distribution by filenames
fig = px.histogram(df, x="Emotions_XLNet", color="Filenames", title="XLNet Emotion Prediction by Filenames",
                   labels={"Emotions_XLNet": "Emotions", "Counts": "Counts"}, width=800, 
                   category_orders={"Emotions_XLNet": df['Emotions_XLNet'].value_counts().index})

fig.update_layout(barmode="group", bargap=0.1, bargroupgap=0, xaxis={'showgrid': False, 'zeroline': False})

fig.show()

In [30]:
# Save the final dataframe to an Excel file
df.to_excel('final_dataframe.xlsx', index=False)
df2 = pd.read_excel("final_dataframe.xlsx")
df2.tail(15)

,Filenames,Texts,Emotions_T5,Confidence_T5,Emotions_BERT,Confidence_BERT,Emotions_XLNet,Confidence_XLNet
338,Audio I,neuroplasticity? And so this is what I study. ...,happiness,0.989409,surprise,0.402655,negetive,0.402628
339,Audio I,recover from stroke and frankly we have failed...,sadness,0.911524,sadness,0.821910,negetive,0.670954
340,Audio I,people recover from stroke and this is an enor...,fear,0.924357,sadness,0.540733,negetive,0.426023
341,Audio I,very expensive problem. So the approach that m...,fear,0.977268,surprise,0.996733,negetive,-0.117016
342,Audio I,makes it very difficult to use statistics to t...,happiness,0.931666,surprise,0.972192,negetive,0.405552
343,Audio I,lesson is that the primary driver of change in...,happiness,0.766965,fear,0.700226,positive,0.049722
344,Audio I,"can work both ways. It can be positive, you le...",happiness,0.985984,surprise,0.631730,negetive,0.596872
345,Audio I,approach to learning. So there's no recipe for...,happiness,0.932310,surprise,0.996323,positive,-0.040509
346,Audio I,forced us to consider something called persona...,happiness,0.969503,surprise,0.655861,negetive,0.576755
347,Audio I,structure and function that we call biomarkers...,happiness,0.989313,surprise,0.927234,negetive,0.524679


In [31]:
# Create and display a 3D scatter plot of all emotion distributions by filenames
fig = px.scatter_3d(df, x='Emotions_T5', y='Emotions_XLNet', z='Emotions_BERT',
                    color='Filenames',title='All Emotion Prediction by Filenames')
fig.show()

In [32]:
#